In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lsst.daf.butler import Butler

import sys, logging
from pathlib import Path
import pandas as pd

proj = Path("/sdf/data/rubin/user/mrakovci/Projects/Asteroid_detection_CNN")

# Make ADCNN importable
sys.path.insert(0, str(proj))

# Make "common.py" (used by simulate_inject.py) importable as a top-level module
sys.path.insert(0, str(proj / "ADCNN" / "data" / "dataset_creation"))

# Your injection code (adjust import path to where ADCNN is on sys.path)
from ADCNN.data.dataset_creation.simulate_inject import one_detector_injection

REPO = "/repo/main"
#COLL = "LSSTComCam/runs/DRP/DP1/w_2025_50/DM-48478"
COLL = "LSSTComCam/runs/DRP/DP1/w_2025_17/DM-50530"
RNG = np.random.default_rng(146)
test_catalog = pd.read_csv("../DATA/test.csv")

In [64]:
_, cat_indexes = np.unique(test_catalog["image_id"].to_numpy(), return_index=True)
visit_detector = test_catalog[["visit", "detector"]].to_numpy()[cat_indexes]
butler = Butler(REPO, collections=COLL)
fp = 0
for visit, detector in visit_detector:
    WHERE = ("instrument='LSSTComCam' "
         "AND skymap='lsst_cells_v1' "
         "AND exposure.observation_type='science' AND band in ('u','g','r','i','z','y') "
         "AND (exposure="+str(visit)+")")
    refs = list(butler.registry.queryDatasets("single_visit_star_footprints", where=WHERE, findFirst=True))
    for i in refs:
        if i.dataId["detector"]==detector:
            ref = i
    LSST_catalog = butler.get("single_visit_star_footprints", dataId=ref.dataId)
    #fp+=len(LSST_catalog)
    fp+=(LSST_catalog["parent"]==0).sum()

tp=692
fn=288
prec = tp / max(tp + fp, 1)
rec  = tp / max(tp + fn, 1)
f1   = 2 * prec * rec / max(prec + rec, 1e-12)
f2  = 5 * tp / max(5 * tp + 4 * fn + fp, 1)
print(f" TP={tp} FP={fp} FN={fn} P={prec:.4f} R={rec:.4f} F1={f1:.4f} F2={f2:.4f}")

 TP=692 FP=86588 FN=288 P=0.0079 R=0.7061 F1=0.0157 F2=0.0379
